# Exercise 2 - Maria Ana P. Guevarra

### Load the Browns corpus from NLTK (nltk.corpus.brown) with fiction category (pass the category to the loader functions). From the corpus, load the tagged and untagged sentences. Make sure that the tags are using the universal tag set. 

### To evaluate the taggers, divide the tagged sentence into 75-25 split for training tagging algorithms and testing them.Report both the accuracy on the training data and testing data.

In [3]:
import nltk
nltk.download('book',quiet=True)

True

In [4]:
data = nltk.corpus.brown.sents()
fiction = nltk.corpus.brown.tagged_sents(categories='fiction',tagset='universal')

In [5]:
split = int(len(fiction)*0.75)
train = fiction[:split]
test = fiction[split:]

In [6]:
from nltk.tag import untag
test_sent = untag(test[0])
print("Tagged: ",test[0])
print("Untagged: ",test_sent)

Tagged:  [('He', 'PRON'), ('had', 'VERB'), ('not', 'ADV'), ('felt', 'VERB'), ('that', 'ADP'), ('during', 'ADP'), ('the', 'DET'), ('afternoon', 'NOUN'), ('.', '.')]
Untagged:  ['He', 'had', 'not', 'felt', 'that', 'during', 'the', 'afternoon', '.']


In [7]:
from nltk import DefaultTagger
print("Accuracy on the training data: %4.1f%%" % (100.0 * DefaultTagger('NUM').evaluate(train)))
print("Accuracy on the testing data: %4.1f%%" % (100.0 * DefaultTagger('NUM').evaluate(test)))

Accuracy on the training data:  0.6%
Accuracy on the testing data:  0.8%


### Submit the notebook that performs the tasks below.

### 1.	Explore the performance of N-Gram taggers on the corpus. 
### a.	Unigram Tagger

In [8]:
unigram_tagger = nltk.tag.UnigramTagger(train)

In [9]:
unigram_tagger.evaluate(test)

0.8438119069961422

In [10]:
unigram_tagger.tag_sents(data)[:5]

[[('The', 'DET'),
  ('Fulton', None),
  ('County', 'NOUN'),
  ('Grand', None),
  ('Jury', None),
  ('said', 'VERB'),
  ('Friday', 'NOUN'),
  ('an', 'DET'),
  ('investigation', None),
  ('of', 'ADP'),
  ("Atlanta's", None),
  ('recent', None),
  ('primary', None),
  ('election', None),
  ('produced', 'VERB'),
  ('``', '.'),
  ('no', 'DET'),
  ('evidence', None),
  ("''", '.'),
  ('that', 'ADP'),
  ('any', 'DET'),
  ('irregularities', None),
  ('took', 'VERB'),
  ('place', 'NOUN'),
  ('.', '.')],
 [('The', 'DET'),
  ('jury', None),
  ('further', 'ADV'),
  ('said', 'VERB'),
  ('in', 'ADP'),
  ('term-end', None),
  ('presentments', None),
  ('that', 'ADP'),
  ('the', 'DET'),
  ('City', 'NOUN'),
  ('Executive', None),
  ('Committee', None),
  (',', '.'),
  ('which', 'DET'),
  ('had', 'VERB'),
  ('over-all', 'ADJ'),
  ('charge', 'NOUN'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('election', None),
  (',', '.'),
  ('``', '.'),
  ('deserves', None),
  ('the', 'DET'),
  ('praise', 'NOUN'),
  ('and'

### b.	Unigram Tagger with a verb backoff

In [11]:
default_tagger = nltk.tag.DefaultTagger('VERB')
unigram_tagger_backoff = nltk.tag.UnigramTagger(train, backoff=default_tagger)

In [12]:
unigram_tagger_backoff.evaluate(train)

0.9633715977771468

In [13]:
unigram_tagger_backoff.evaluate(test)

0.8678448545511417

### c.	Trigram Tagger with Unigram Tagger and adjective backoff

In [14]:
default_tagger = nltk.tag.DefaultTagger('ADJ')
unigram_tagger = nltk.tag.UnigramTagger(train)
trigram_tagger_backoff = nltk.tag.TrigramTagger(train,backoff=default_tagger)

In [15]:
trigram_tagger_backoff.evaluate(train)

0.9177382063034925

In [16]:
trigram_tagger_backoff.evaluate(test)

0.4365551037430925

### d.	Trigram Tagger with a Bigram Tagger backoff

In [17]:
bigram_tagger = nltk.tag.BigramTagger(train)
trigram_tagger_backoff = nltk.tag.UnigramTagger(train, backoff=bigram_tagger)

In [18]:
trigram_tagger_backoff.evaluate(train)

0.9340242566827567

In [19]:
trigram_tagger_backoff.evaluate(test)

0.4915545824210197

### 2.	Train an Average Perceptron Tagger with different iterations. Compare the results of using different iterations.
### a.	1 iteration

In [20]:
perceptron_trained = nltk.perceptron.PerceptronTagger(load=False)
perceptron_trained.train(train,nr_iter=1)

In [21]:
perceptron_trained.evaluate(train)

0.9591327627469274

In [22]:
perceptron_trained.evaluate(test)

0.9283182149932229

### b.	5 iteration

In [27]:
perceptron_trained = nltk.perceptron.PerceptronTagger(load=False)
perceptron_trained.train(train,nr_iter=5)

In [28]:
perceptron_trained.evaluate(train)

0.9951324382428102

In [29]:
perceptron_trained.evaluate(test)

0.94906683348973

### c.	10 iteration

In [30]:
perceptron_trained = nltk.perceptron.PerceptronTagger(load=False)
perceptron_trained.train(train,nr_iter=10)

In [31]:
perceptron_trained.evaluate(train)

0.9990264876485621

In [32]:
perceptron_trained.evaluate(test)

0.9524032947554999

## Based on the data above, more iterations produce much more accurate result.

### Train a 3 Conditional Random Field using a different custom feature function. The feature function must contain the features below. Model A should use features a-c. Model B should use features a-e and Model C should use all the features.

### a. Previous, Current, and Next Word
### b. 1-3 Character Prefix
### c. 1-3 Character Suffix
### d. Capitalize
### e. Word contains a number
### f. Word is first in the sentence
### g. Word is last in the sentence

### Model A

In [33]:
def modelA_features(sentence, index):
    return {
        'word': sentence[index],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:]
    }

In [34]:
crf_custom_A = nltk.crf.CRFTagger(feature_func=modelA_features)
crf_custom_A.train(train,'crf_custom_A.tag')

In [35]:
crf_modelA_train = crf_custom_A.evaluate(train)
print("Training data accuracy: ", crf_modelA_train)

Training data accuracy:  0.9786232912830082


In [36]:
crf_modelA_test = crf_custom_A.evaluate(test)
print("Testing data accuracy: ", crf_modelA_test)

Testing data accuracy:  0.9516213116463351


### Model B

In [37]:
def modelB_features(sentence, index):
    return {
        'word': sentence[index],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        
   
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        

        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        

        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        
   
        'is_numeric': sentence[index].isdigit()
    }

In [38]:
crf_custom_B = nltk.crf.CRFTagger(feature_func=modelB_features)
crf_custom_B.train(train, 'crf_custom_B.tag')

In [39]:
crf_modelB_train = crf_custom_B.evaluate(train)
print("Training data accuracy: ", crf_modelB_train)

Training data accuracy:  0.9785827282683649


In [40]:
crf_modelB_test = crf_custom_B.evaluate(test)
print("Testing data accuracy: ", crf_modelB_test)

Testing data accuracy:  0.9516734438536127


### Model C

In [41]:
def modelC_features(sentence, index):
    return {
        'word': sentence[index],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        
    
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        

        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        
 
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        
 
        'is_numeric': sentence[index].isdigit(),
        

        'is_first': index == 0,
        

        'is_last': index == len(sentence) - 1
    }

In [42]:
crf_custom_C = nltk.crf.CRFTagger(feature_func=modelC_features)
crf_custom_C.train(train, 'crf_custom_C.tag')

In [43]:
crf_modelC_train = crf_custom_C.evaluate(train)
print("Training data accuracy: ", crf_modelC_train)

Training data accuracy:  0.9785421652537216


In [44]:
crf_modelC_test = crf_custom_C.evaluate(test)
print("Testing data accuracy: ", crf_modelC_test)

Testing data accuracy:  0.9516734438536127
